<a href="https://colab.research.google.com/github/kterashi/niigata_lecture_2024/blob/master/Lec4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulation of Quantum Dynamics

$\newcommand{\bra}[1]{|#1\rangle}$
$\newcommand{\ket}[1]{|#1\rangle}$
$\newcommand{\braket}[2]{\langle #1 | #2 \rangle}$
$\newcommand{\upket}{\ket{\uparrow}}$
$\newcommand{\downket}{\ket{\downarrow}}$
$\newcommand{\rightket}{\ket{\rightarrow}}$
$\newcommand{\plusket}{\ket{+}}$
$\newcommand{\minusket}{\ket{-}}$

In [ ]:
# First, get all the necessary libraries from the copy and import packages
import os
import sys
import shutil
import tarfile
from google.colab import drive
drive.mount('/content/gdrive')
shutil.copy('/content/gdrive/MyDrive/qcintro.tar.gz', '.')
with tarfile.open('qcintro.tar.gz', 'r:gz') as tar:
    tar.extractall(path='/root/.local')

sys.path.append('/root/.local/lib/python3.10/site-packages')

In [ ]:
from typing import Optional
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile
from qiskit.primitives import SamplerResult
from qiskit.quantum_info import SparsePauliOp
from qiskit.result import Result
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import Estimator as AerEstimator, Sampler as AerSampler
from qiskit_ibm_runtime import Sampler as RuntimeSampler, Options, QiskitRuntimeService, Session
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

runtime_config_path = '/content/gdrive/MyDrive/qiskit-ibm.json'

## Mapping a single spin-1/2 system onto a qubit

Single spin-1/2 system can be trivially expressed with a qubit. In the next cells we perform a (first-order) Suzuki-Trotter simulation of the evolution under a time-dependent Hamiltonian

$$
H(t) = \omega \ket{\uparrow}\bra{\uparrow} + \frac{A}{2} \cos{\alpha t} \left(\ket{\uparrow}\bra{\downarrow} + \ket{\downarrow}\ket{\uparrow}\right)
= H_Z + H_X(t).
$$

First we define the functions that return the circuits for single Trotter steps:

In [ ]:
def uz_spin(omega: float, delta_t: float) -> QuantumCircuit:
    """Short-time evolution by exp(-i H_Z Δt)."""
    circ = QuantumCircuit(1)
    # exp(-i ω |↑><↑| Δt) <=> exp(-i/2 ω Δt Z)
    circ.rz(omega * delta_t, 0)
    return circ

def ux_spin(amp: float, alpha_t: float, delta_t: float) -> QuantumCircuit:
    """Short-time evolution by exp(-i H_X(t) Δt)."""
    circ = QuantumCircuit(1)
    # exp(-i A/2 cos(αt) (|↑><↓|+|↓><↑|) Δt) <=> exp(-i/2 A cos(αt) Δt X)
    circ.rx(amp * np.cos(alpha_t) * delta_t, 0)
    return circ

Then the function to construct the full simulation circuit:

In [ ]:
def driven_spin_simulation_sv(
    omega: float,
    amp: float,
    alpha: float,
    sim_time: float,
    num_steps: int
) -> QuantumCircuit:
    """Suzuki-Trotter simulation of the evolution by H(t) using statevectors."""
    delta_t = sim_time / num_steps
    # Single qubit <=> single spin 1/2
    circ = QuantumCircuit(1)
    for istep in range(num_steps):
        # Instruction to the simulator to save the statevector snapshot at this point in the circuit
        circ.save_statevector(label=f'step{istep}')

        # Append the H_Z and H_X(t) simulation circuits to the main circuit
        circ.compose(uz_spin(omega, delta_t), inplace=True)
        circ.compose(ux_spin(amp, alpha * (sim_time / num_steps * istep), delta_t), inplace=True)

    # Instruction to the simulator to save the final state
    circ.save_statevector(label=f'step{num_steps}')

    return circ

And a visualization function:

In [ ]:
def plot_bloch_sv(sim_time: float, sim_result: Result):
    """Visualization of the statevector data as time series of X, Y, Z Bloch coordinates."""
    num_steps = len(sim_result.data()) - 1
    ts = np.linspace(0., sim_time, num_steps + 1)
    xs = np.empty_like(ts)
    ys = np.empty_like(ts)
    zs = np.empty_like(ts)

    # Pauli matrices
    x = np.array([[0., 1.], [1., 0.]], dtype=complex)
    y = np.array([[0., -1.j], [1.j, 0.]], dtype=complex)
    z = np.array([[1., 0.], [0., -1.]], dtype=complex)

    # Compute <X>, <Y>, and <Z> of each statevector
    for istep in range(num_steps + 1):
        sv = sim_result.data()[f'step{istep}'].data
        xs[istep] = (sv.conjugate() @ x @ sv).real
        ys[istep] = (sv.conjugate() @ y @ sv).real
        zs[istep] = (sv.conjugate() @ z @ sv).real
    
    plt.plot(ts, xs, label='X')
    plt.plot(ts, ys, label='Y')
    plt.plot(ts, zs, label='Z')
    plt.legend();

Construct the simulator and run the simulation for several values of the parameters:

In [ ]:
sv_simulator = AerSimulator(method='statevector')

In [ ]:
# Off-resonant driving
omega = 2. * np.pi
amp = 0.1
alpha = 0.2 * np.pi
sim_time = 10.
num_steps = 100

circuit = driven_spin_simulation_sv(omega, amp, alpha, sim_time, num_steps)
sim_result = sv_simulator.run(circuit).result()
plot_bloch_sv(sim_time, sim_result)

In [ ]:
# On-resonant driving
omega = 2. * np.pi
amp = 0.5
alpha = 2. * np.pi
sim_time = 20.
num_steps = 200

circuit = driven_spin_simulation_sv(omega, amp, alpha, sim_time, num_steps)
sv_result = sv_simulator.run(circuit).result()
plot_bloch_sv(sim_time, sv_result)

If we were to not cheat with the statevectors, the simulation function would instead look like this:

In [ ]:
def driven_spin_simulation_expval(
    omega: float,
    amp: float,
    alpha: float,
    sim_time: float,
    num_steps: int
) -> tuple[list[QuantumCircuit], list[SparsePauliOp]]:
    """Estimator inputs for Suzuki-Trotter simulation of the evolution by H(t)."""
    delta_t = sim_time / num_steps
    circuits = []
    observables = []
    # Single qubit <=> single spin 1/2
    circ = QuantumCircuit(1)
    for istep in range(num_steps):
        # Store one copy (snapshot) of circ per observable
        for obs in ['X', 'Y', 'Z']:
            circuits.append(circ.copy())
            observables.append(SparsePauliOp(obs))
            
        # Append the H_Z and H_X(t) simulation circuits to the main circuit
        circ.compose(uz_spin(omega, delta_t), inplace=True)
        circ.compose(ux_spin(amp, alpha * (sim_time / num_steps * istep), delta_t), inplace=True)

    # Final-state snapshots
    for obs in ['X', 'Y', 'Z']:
        circuits.append(circ)
        observables.append(SparsePauliOp(obs))

    return circuits, observables

In [ ]:
def plot_bloch_expval(sim_time: float, expvals: np.ndarray):
    """Visualization of the X, Y, Z expectation values as functions of time."""
    num_steps = expvals.shape[0] // 3 - 1
    ts = np.linspace(0., sim_time, num_steps + 1)
    xs = expvals[::3]
    ys = expvals[1::3]
    zs = expvals[2::3]
    
    plt.plot(ts, xs, label='X')
    plt.plot(ts, ys, label='Y')
    plt.plot(ts, zs, label='Z')
    plt.legend();

In [ ]:
circuits, observables = driven_spin_simulation_expval(omega, amp, alpha, sim_time, num_steps)
estimator = AerEstimator()
expval_result = estimator.run(circuits, observables, shots=2000, seed=1234).result()

In [ ]:
plot_bloch_expval(sim_time, expval_result.values)

To run the same circuits on an IBM backend, we would in principle need to simply replace the AerEstimator with RuntimeEstimator set up with an appropriate backend. However, to be economical, we will try to "parallelize" the circuits using as many qubits as possible. Because a single circuit will be used to compute observables of multiple independent experiments, we cannot use the Estimator and will instead be using the Sampler. Change of measurement basis and expectation value extraction are performed by hand.

In [ ]:
def driven_spin_simulation_parallel(
    omega: float,
    amp: float,
    alpha: float,
    sim_time: float,
    num_steps: int,
    num_qubits: int
) -> list[QuantumCircuit]:
    """Parallelized Suzuki-Trotter simulation of the evolution by H(t)."""
    # Circuits to be run (num_qubits dense)
    circuits = [QuantumCircuit(num_qubits)]
    # Function to yield the next qubit index to append the simulation circuit
    # When all qubits of the current circuit is exhausted, appends a new circuit
    # to the list and restarts the qubit index from 0
    def gen_qubit_idx():
        while True:
            for iq in range(num_qubits):
                yield iq
            # Exhausted all qubits; append a new one to the list
            circuits.append(QuantumCircuit(num_qubits))

    # Qubit index generator
    qubit_idx = gen_qubit_idx()
            
    delta_t = sim_time / num_steps
    # Simulation (single-qubit) circuit
    circ = QuantumCircuit(1)
    for istep in range(num_steps):
        # Append the single-qubit circuit to the next qubit of the output circuit
        for obs in ['X', 'Y', 'Z']:
            iq = next(qubit_idx)
            circuits[-1].compose(circ, qubits=[iq], inplace=True)
            # Apply change of measurement basis
            if obs == 'X':
                circuits[-1].h(iq)
            elif obs == 'Y':
                circuits[-1].sdg(iq)
                circuits[-1].h(iq)

        # Append the H_Z and H_X(t) simulation circuits to the simulation circuit
        circ.compose(uz_spin(omega, delta_t), inplace=True)
        circ.compose(ux_spin(amp, alpha * (sim_time / num_steps * istep), delta_t), inplace=True)

    # Final-state snapshots
    for obs in ['X', 'Y', 'Z']:
        iq = next(qubit_idx)
        circuits[-1].compose(circ, qubits=[iq], inplace=True)
        # Apply change of measurement basis
        if obs == 'X':
            circuits[-1].h(iq)
        elif obs == 'Y':
            circuits[-1].sdg(iq)
            circuits[-1].h(iq)

    # Using the Sampler -> need explicit measurement instructions
    for circuit in circuits:
        circuit.measure_all()

    return circuits

def extract_expvals(num_steps: int, num_qubits: int, result: SamplerResult):
    """Function to extract the expectation values from a Sampler result (quasi-distributions)."""
    # Temporary array with shape [N, 2] (N: total number of simulation circuits)
    qvals = np.zeros((3 * (num_steps + 1), 2), dtype=float)

    # result.quasi_dists is a list of dict-like objects (one per executed circuit)
    for idist, dist in enumerate(result.quasi_dists):
        # Range of simulation circuit indices this result corresponds to
        min_index = idist * num_qubits
        max_index = min((idist + 1) * num_qubits, qvals.shape[0])
        indices = np.arange(min_index, max_index)
        # binary_probabilities = {bitstring: quasi-probability}
        for bitstring, quasip in dist.binary_probabilities(num_qubits).items():
            # Bits are ordered from right to left in the string
            bits = [int(s) for s in bitstring[::-1]]
            # Sum up the quasi-probability of observing 0 or 1 for each simulation circuit
            qvals[indices, bits[:len(indices)]] += quasip

    # Finally compute the expectation value as a difference between qvals[:, 0] and qvals[:, 1]
    return qvals[:, 0] - qvals[:, 1]

Trying the parallelized version on the simulator first:

In [ ]:
num_qubits = 16
circuits = driven_spin_simulation_parallel(omega, amp, alpha, sim_time, num_steps, num_qubits)
sampler = AerSampler()
par_result = sampler.run(circuits, shots=2000, seed=1234).result()
expvals = extract_expvals(num_steps, num_qubits, par_result)

In [ ]:
plot_bloch_expval(sim_time, expvals)

And then finally on a backend:

In [ ]:
service = QiskitRuntimeService(filename=runtime_config_path)
backend = service.least_busy(operational=True, simulator=False)

In [ ]:
# List of usable qubits on the backend
qubits = [iq for iq in range(backend.num_qubits) if iq not in backend.properties().faulty_qubits()]
num_qubits = len(qubits)
circuits = driven_spin_simulation_parallel(omega, amp, alpha, sim_time, num_steps, num_qubits)

# Submitting un-transpiled circuits will soon be forbidden, so we transpile ours locally
#circuits_tr = transpile(circuits, backend=backend, initial_layout=qubits, optimization_level=0)
pm = generate_preset_pass_manager(backend=backend, initial_layout=qubits, optimization_level=0)
circuits_tr = pm.run(circuits)

# For now we need to set the skip_transpilation option to signal that the circuits have been transpiled
# Resilience level 1 => perform readout error mitigation
#options = Options(resilience_level=1, transpilation={'skip_transpilation': True})
with Session(backend=backend) as session:
    sampler = RuntimeSampler(mode=session)
    ibm_result = sampler.run(circuits_tr, shots=2000).result()

In [ ]:
ibm_expvals = extract_expvals(num_steps, num_qubits, ibm_result)
plot_bloch_expval(sim_time, ibm_expvals)